## Setup

In this notebook, we consider solving a simple structural mechanics PDE using our finite element solver. In particular, we consider the simple linear static Hookean response modeled as:

$$
\nabla \cdot \boldsymbol{\sigma} + \mathbf{f} = 0
\qquad \text{in } \Omega,
$$

$$
\mathbf{u} = \mathbf{u}_{0}
\qquad \text{on } \Gamma_{u},
$$

$$
-\,\boldsymbol{\sigma} \cdot \mathbf{n} = \mathbf{t}_{0}
\qquad \text{on } \Gamma_{t}.
$$
Where:
- **σ** is the Cauchy stress tensor,
- **f** is the body force per unit volume,
- **u** is the displacement vector field,
- **u**₀ is the prescribed displacement on the Dirichlet boundary Γᵤ,
- **t**₀ is the prescribed traction on the Neumann boundary Γₜ,
- **n** is the outward unit normal vector to the boundary.

Furthermore, the constitutive relation for a Hookean material can be described as:

$$
\boldsymbol{\sigma}
  = \lambda\,\operatorname{tr}(\boldsymbol{\varepsilon})\,\mathbf{I}
    + 2\mu\,\boldsymbol{\varepsilon}
$$

$$
\boldsymbol{\varepsilon}
  = \tfrac{1}{2}\bigl(\nabla\mathbf{u} + (\nabla\mathbf{u})^{\mathsf{T}}\bigr).
$$
Where:
- **ε** is the strain tensor.
- λ and μ are the Lamé's first and second parameters.
- **I** is the identity tensor.

To illustrate this, we consider a simple L-beam problem, as shown in the figure below:

<img src="../../figures/l_bracket.png" alt="L-Bracket" style="width: 200px;"/>

This notebook describes the problem setup for a simulation using a bilinear quadrilateral mesh.

Finally, we compare the results with those obtained from Ansys to validate our solver.


In [ ]:
import numpy as np
import jax.numpy as jnp
import jax
import matplotlib.pyplot as plt

import toflux.src.geometry as _geom
import toflux.src.mesher as _mesher
import toflux.src.material as _mat
import toflux.src.bc as _bc

import toflux.src.fe_struct as _fea
import toflux.src.solver as _solv
import toflux.src.viz as _viz

_Disp = _fea.DisplacementField

jax.config.update("jax_enable_x64", True)

plt.rcParams.update(_viz.high_res_plot_settings)

## Mesh
Below we set up the classical **L-bracket** benchmark that is widely used in structural analysis and topology-optimization.

* **Geometry**  
  The outline is stored in a `GeoJSON` file and imported directly into the mesher.

* **Discretisation**  
  The domain $\Omega$ is subdivided into bilinear quadrilateral (voxel) elements.  
  Each node carries two mechanical degrees of freedom: the displacements $u_x$ and $u_y$.

* **Numerical integration**  
  Element matrices are evaluated with a $3\times3$ Gauss quadrature, providing third-order accuracy for the stiffness calculation.

This mesh forms the starting point for both the finite-element analysis and any subsequent topology-optimization runs.

In [ ]:
geom = _geom.BrepGeometry("toflux/brep/Lbeam.json")

mesh = _mesher.grid_mesh_brep(
  brep=geom,
  nelx_desired=80,
  nely_desired=80,
  dofs_per_node=2,
  gauss_order=3,
)

density = jnp.zeros(mesh.num_elems)
_viz.plot_grid_mesh(mesh, density)

In [ ]:
_viz.plot_brep(geom)

## Material


We consider a simplified scenario where the Young's modulus is unity. Furthermore, we consider a small deformations. 

In [ ]:
material_params = _mat.StructuralMaterial(
  youngs_modulus=1.0,
  poissons_ratio=0.30,
)

deformation_model = _fea.DeformationModel.SMALL

## Boundary conditions

As illustrated in the Figure, we fix the top edge with a load applied on the hanging face. We find the faces (and nodes) associated with the boundary condition. We provide a simplified boundary condition module where the specified values are applied directly on the nodes. The boundary conditions are tabulated as:
| Part of geometry | Type | Specification |
|------------------|------|---------------|
| **Top horizontal edge** | **Dirichlet (fixed support)** | \(u_x = u_y = 0\) |
| **Hanging edge**  | **Neumann (surface traction)** | Downward load |

In [ ]:
hang_faces = _bc.identify_faces(mesh, edges=[geom.edges[2]])
n = len(hang_faces)
hang_face_val = [(_Disp.V, -5.e-4 * jnp.ones(n))]


top_faces = _bc.identify_faces(mesh, edges=[geom.edges[5]])
n = len(top_faces)
xv = (_Disp.U, jnp.zeros(n))
yv = (_Disp.V, jnp.zeros(n))

top_face_val = [xv, yv]

fixed_bc = _bc.DirichletBC(elem_faces=top_faces, values=top_face_val, name="fix")
load_bc = _bc.NeumannBC(elem_faces=hang_faces, values=hang_face_val, name="load")

bc_list = [fixed_bc, load_bc]
bc = _bc.process_boundary_conditions(bc_list, mesh)

_viz.plot_bc(bc_list, mesh)

## Solver

We define the settings for the solver. By default, all our FEA solvers are programmed as a subclass of nonlinear problems. Hence, the solver expects the settings for both the linear and nonlinear solvers.
Even when the physics is linear, the solver still performs a Newton–Raphson step; for a small-strain model this step converges in **one iteration**. However, when choosing a large deformation model, the problem is nonlinear and more newton raphson iterations might be needed. Furthermore, we have interfaced JAX with a variety of fast linear solvers whose appropriate settings needs to be passed. For more details see `solver.py`.

In [ ]:
solver_settings = {
  "linear": {"solver": _solv.LinearSolvers.SCIPY_SPARSE, "scipy_solver": {}},
  "nonlinear": {"max_iter": 1, "threshold": 1.0e-4},
}

## FEA initalize

We initialize the FEA solver with the mesh, material, boundary conditions and solver settings. Additionally, for structural simulation, we offer a small or large (nonlinear) deformation model for the user to experiment.

In [ ]:
fea = _fea.FEA(
  mesh=mesh,
  material=material_params,
  deformation_model=deformation_model,
  bc=bc,
  solver_settings=solver_settings,
)

## Solve

We associate each element with the Lame's parameters. Furthermore, we provide an inital guess of the solution with the Dirichlet condition enforced.

In [ ]:
lam, mu = material_params.lame_parameters
lam = lam * jnp.ones(mesh.num_elems)
mu = mu * jnp.ones(mesh.num_elems)

u0 = jnp.zeros((mesh.num_dofs,)) + 1.e-6
u0 = u0.at[bc['fixed_dofs']].set(bc['dirichlet_values'])

u = _solv.newton_raphson_solve(fea, u0, lam, mu)

## Plot

We visualize the dsplacement magnitude here.

In [ ]:
node_deformation = np.stack((u[0::2], u[1::2]), axis=1)
deformed_mesh = _mesher.deform_mesh(fea.mesh, node_deformation)

u_mag = jnp.linalg.norm(node_deformation, axis=1)
u_elem = u_mag[mesh.elem_nodes].mean(axis=1)

fig, ax = plt.subplots(1, 1)
ax = _viz.plot_grid_mesh(
  deformed_mesh,
  u_elem,
  ax=ax,
  colorbar=True,
)